In [1]:
import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek

# 加载环境变量
load_dotenv()
api = os.getenv("DEEPSEEK_API_KEY")
print("api = ", api)

api =  None


In [2]:
markdown_path = "../data/C1/markdown/easy-rl-chapter1.md"
loader = TextLoader(markdown_path)
docs = loader.load()


In [3]:
text_splitter = RecursiveCharacterTextSplitter()
texts = text_splitter.split_documents(docs)


In [4]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)


/home/codespace/miniconda3/envs/all-in-rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
vectorstore = InMemoryVectorStore(embeddings)
vectorstore.add_documents(texts)


['d564398c-0428-4c32-94b4-4156018eee26',
 '4ebf1481-cb9b-43e1-a0bb-da40cc8125f4',
 '4fcd13fc-5013-466b-a358-7960cfadadd3',
 'f0743a48-64d8-4493-8300-81bb6be0e062',
 '46c64671-a9ea-4143-967a-08c61b576082',
 'd564302a-cd98-487b-bf4e-cd98a6c1b74e',
 '8ff4fc1f-5ffd-418c-bc58-91777eb4b068']

In [6]:
question = "文中举了哪些例子？"


In [7]:
retrieved_docs = vectorstore.similarity_search(question, k=3)


In [8]:
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)


In [9]:
prompt = ChatPromptTemplate.from_template("""请根据下面提供的上下文信息来回答问题。
请确保你的回答完全基于这些上下文。
如果上下文中没有足够的信息来回答问题，请直接告知：“抱歉，我无法根据提供的上下文找到相关信息来回答此问题。”
  
上下文:
{context}

问题: {question}

回答:"""
                                          )


In [10]:
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.7,
    max_tokens=2048,
    api_key=os.getenv("DEEPSEEK_API_KEY")
)


ValidationError: 1 validation error for ChatDeepSeek
  Value error, If using default api base, DEEPSEEK_API_KEY must be set. [type=value_error, input_value={'model': 'deepseek-chat'...one, 'model_kwargs': {}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
answer = llm.invoke(prompt.format(question=question, context=docs_content))
print(answer)
